ETL PROCESS: HR Analytics - Job Change of Data Scentists

In [90]:
import pandas as pd
from sqlalchemy import create_engine

def read_data(source_type, path=None, user_name=None, pw=None, server=None, port=None, db_name=None, table_name=None, table_index=None):


    if source_type == 'ggsheet':
        url = f"https://docs.google.com/spreadsheets/d/{path}/export?format=csv"
        return pd.read_csv(url)

    elif source_type == 'csv':
        return pd.read_csv(path)

    elif source_type == 'excel':
        return pd.read_excel(path)

    elif source_type == 'sql':
        engine = create_engine('mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.format(user_name,pw,server,port,db_name))
        return pd.read_sql_table(table_name, con=engine)

    elif source_type == 'html':
        tables = pd.read_html(path)
        return tables[table_index if table_index is not None else 0]

    else:
        raise ValueError("Unsupported source type.")


In [91]:
# Install the pymysql library to enable MySQL database connections in Python
!pip install pymysql

In [92]:
# Install the sqlalchemy library for SQL toolkit and Object-Relational Mapping (ORM) in Python
!pip install sqlalchemy

In [93]:
# Import pymysql for MySQL database connections
import pymysql

# INTRODUCTION
## Context and Content
A company which is active in Big Data and Data Science wants to hire data scientists among people who successfully pass some courses which conduct by the company.

Many people signup for their training. Company wants to know which of these candidates are really wants to work for the company after training or looking for a new employment because it helps to reduce the cost and time as well as the quality of training or planning the courses and categorization of candidates.

Information related to demographics, education, experience are in hands from candidates signup and enrollment.

  # DATA SOURCE

##  1. Enrollies' data

As enrollies are submitting their request to join the course via Google Forms, we have the Google Sheet that stores data about enrolled students, containing the following columns:

- enrollee_id: unique ID of an enrollee
- full_name: full name of an enrollee
- city: the name of an enrollie's city
- gender: gender of an enrolleeThe source: https://docs.google.com/spreadsheets/d/1VCkHwBjJGRJ21asd9pxW4_0z2PWuKhbLR3gUHm-p4GI/edit?usp=sharing

In [94]:
# Read the Excel file from the Google Sheet URL into a Pandas DataFrame
enrollies_data = read_data('ggsheet','1VCkHwBjJGRJ21asd9pxW4_0z2PWuKhbLR3gUHm-p4GI')

In [95]:
# Display a summary of the DataFrame, including the number of non-null entries, column types
enrollies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   enrollee_id  19158 non-null  int64 
 1   full_name    19158 non-null  object
 2   city         19158 non-null  object
 3   gender       14650 non-null  object
dtypes: int64(1), object(3)
memory usage: 598.8+ KB


In [96]:
# Display the first 5 rows of the DataFrame to quickly preview the data
enrollies_data.head()

,enrollee_id,full_name,city,gender
0,8949,Mike Jones,city_103,Male
1,29725,Laura Jones,city_40,Male
2,11561,David Miller,city_21,NaN
3,33241,Laura Davis,city_115,NaN
4,666,Alex Martinez,city_162,Male


In [97]:
# Replace missing values (NaN) in the 'gender' column with the string 'unknown'
enrollies_data['gender'] = enrollies_data['gender'].fillna('unknown')

In [98]:
# Display all unique values in the 'gender' column
enrollies_data['gender'].unique()

array(['Male', 'unknown', 'Female', 'Other'], dtype=object)

In [99]:
# Display all unique values in the 'city' column
enrollies_data['city'].unique()

array(['city_103', 'city_40', 'city_21', 'city_115', 'city_162',
       'city_176', 'city_160', 'city_46', 'city_61', 'city_114',
       'city_13', 'city_159', 'city_102', 'city_67', 'city_100',
       'city_16', 'city_71', 'city_104', 'city_64', 'city_101', 'city_83',
       'city_105', 'city_73', 'city_75', 'city_41', 'city_11', 'city_93',
       'city_90', 'city_36', 'city_20', 'city_57', 'city_152', 'city_19',
       'city_65', 'city_74', 'city_173', 'city_136', 'city_98', 'city_97',
       'city_50', 'city_138', 'city_82', 'city_157', 'city_89',
       'city_150', 'city_70', 'city_175', 'city_94', 'city_28', 'city_59',
       'city_165', 'city_145', 'city_142', 'city_26', 'city_12',
       'city_37', 'city_43', 'city_116', 'city_23', 'city_99', 'city_149',
       'city_10', 'city_45', 'city_80', 'city_128', 'city_158',
       'city_123', 'city_7', 'city_72', 'city_106', 'city_143', 'city_78',
       'city_109', 'city_24', 'city_134', 'city_48', 'city_144',
       'city_91', 'city_

In [100]:
# Automatically infer and convert the data types of all columns to the most appropriate types
enrollies_data = enrollies_data.convert_dtypes()

In [101]:
# Double-check the data after cleaning
enrollies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   enrollee_id  19158 non-null  Int64 
 1   full_name    19158 non-null  string
 2   city         19158 non-null  string
 3   gender       19158 non-null  string
dtypes: Int64(1), string(3)
memory usage: 617.5 KB


## 2. Enrollies' education

After enrollment everyone should fill the form about their education level. This form is being digitalized manually. Educational department stores it in the Excel format here: https://assets.swisscoding.edu.vn/company_course/enrollies_education.xlsx

This table contains the following columns:

- enrollee_id: A unique identifier for each enrollee. This integer value uniquely distinguishes each participant in the dataset.

- enrolled_university: Indicates the enrollee's university enrollment status. Possible values include no_enrollment, Part time course, and Full time course.

- education_level: Represents the highest level of education attained by the enrollee. Examples include Graduate, Masters, etc.

- major_discipline: Specifies the primary field of study for the enrollee. Examples include STEM, Business Degree, etc.

In [102]:
# Read the 'enrollies_education.xlsx' file from the specified path into a Pandas DataFrame
enrollies_education = read_data('excel','/content/enrollies_education.xlsx')

In [103]:
# Display a summary of the DataFrame, including the number of non-null entries, column types
enrollies_education.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   enrollee_id          19158 non-null  int64 
 1   enrolled_university  18772 non-null  object
 2   education_level      18698 non-null  object
 3   major_discipline     16345 non-null  object
dtypes: int64(1), object(3)
memory usage: 598.8+ KB


In [104]:
# Display the first 5 rows of the DataFrame to quickly preview the data
enrollies_education.head()

,enrollee_id,enrolled_university,education_level,major_discipline
0,8949,no_enrollment,Graduate,STEM
1,29725,no_enrollment,Graduate,STEM
2,11561,Full time course,Graduate,STEM
3,33241,NaN,Graduate,Business Degree
4,666,no_enrollment,Masters,STEM


In [105]:
# Replace missing values in the 'enrolled_university', 'education_level', 'major_discipline' column with the most frequent value (mode) of that column
enrollies_education['enrolled_university'] = enrollies_education['enrolled_university'].fillna(enrollies_education['enrolled_university'].mode()[0])
enrollies_education['education_level'] = enrollies_education['education_level'].fillna(enrollies_education['education_level'].mode()[0])
enrollies_education['major_discipline'] = enrollies_education['major_discipline'].fillna(enrollies_education['major_discipline'].mode()[0])

In [106]:
# Display all unique values in the 'enrolled_university' column to check the distinct categories
enrollies_education['enrolled_university'].unique()

array(['no_enrollment', 'Full time course', 'Part time course'],
      dtype=object)

In [107]:
# Replace 'no_enrollment' with 'no enrollment' and convert all text in the 'enrolled_university' column to lowercase
enrollies_education['enrolled_university'].str.replace('no_enrollment','no enrollment').str.lower()

,enrolled_university
0,no enrollment
1,no enrollment
2,full time course
3,no enrollment
4,no enrollment
...,...
19153,no enrollment
19154,no enrollment
19155,no enrollment
19156,no enrollment


In [108]:
# Display all unique values in the 'education_level' column to check the distinct categories
enrollies_education['education_level'].unique()

array(['Graduate', 'Masters', 'High School', 'Phd', 'Primary School'],
      dtype=object)

In [109]:
# Display all unique values in the 'major_discipline' column to check the distinct categories
enrollies_education['major_discipline'].unique()

array(['STEM', 'Business Degree', 'Arts', 'Humanities', 'No Major',
       'Other'], dtype=object)

In [110]:
# Convert all text in the 'major_discipline' column to uppercase
enrollies_education['major_discipline'].str.upper()

,major_discipline
0,STEM
1,STEM
2,STEM
3,BUSINESS DEGREE
4,STEM
...,...
19153,HUMANITIES
19154,STEM
19155,STEM
19156,STEM


In [111]:
# Automatically infer and convert the data types of all columns to the most appropriate types
enrollies_education = enrollies_education.convert_dtypes()

In [112]:
# Double-check the data after cleaning
enrollies_education.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   enrollee_id          19158 non-null  Int64 
 1   enrolled_university  19158 non-null  string
 2   education_level      19158 non-null  string
 3   major_discipline     19158 non-null  string
dtypes: Int64(1), string(3)
memory usage: 617.5 KB


##  3. Enrollies' working experience

Another survey that is being collected manually by educational department is about working experience.

Educational department stores it in the CSV format here: https://assets.swisscoding.edu.vn/company_course/work_experience.csv

This table contains the following columns:

- enrollee_id: A unique identifier for each enrollee. This integer value uniquely distinguishes each participant in the dataset.

- relevent_experience: Indicates whether the enrollee has relevant work experience related to the field they are currently studying or working in. Possible values include Has relevent experience and No relevent experience.

- experience: Represents the number of years of work experience the enrollee has. This can be a specific number or a range (e.g., >20, <1).

- company_size: Specifies the size of the company where the enrollee has worked, based on the number of employees. Examples include 50−99, 100−500, etc.

- company_type: Indicates the type of company where the enrollee has worked. Examples include Pvt Ltd, Funded Startup, etc.

- last_new_job: Represents the number of years since the enrollee's last job change. Examples include never, >4, 1, etc.

In [113]:
# Read the 'work_experience.csv' file from the specified path into a Pandas DataFrame
working_experience = read_data('csv','/content/work_experience.csv')

In [114]:
# Display a summary of the DataFrame, including the number of non-null entries, column types
working_experience.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   enrollee_id          19158 non-null  int64 
 1   relevent_experience  19158 non-null  object
 2   experience           19093 non-null  object
 3   company_size         13220 non-null  object
 4   company_type         13018 non-null  object
 5   last_new_job         18735 non-null  object
dtypes: int64(1), object(5)
memory usage: 898.2+ KB


In [115]:
# Display the first 5 rows of the DataFrame to quickly preview the data
working_experience.head()

,enrollee_id,relevent_experience,experience,company_size,company_type,last_new_job
0,8949,Has relevent experience,>20,NaN,NaN,1
1,29725,No relevent experience,15,50-99,Pvt Ltd,>4
2,11561,No relevent experience,5,NaN,NaN,never
3,33241,No relevent experience,<1,NaN,Pvt Ltd,never
4,666,Has relevent experience,>20,50-99,Funded Startup,4


In [116]:
# Replace missing values in the 'experience' column with the most frequent value (mode) of that column
working_experience['experience'] = working_experience['experience'].fillna(working_experience['experience'].mode()[0])
# Replace missing values in the 'company_size' column with the string 'unknown'
working_experience['company_size'] = working_experience['company_size'].fillna('unknown')
# Replace missing values in the 'company_type' column with the string 'unknown'
working_experience['company_type'] = working_experience['company_type'].fillna('unknown')
# Replace missing values in the 'last_new_job' column with the most frequent value (mode) of that column
working_experience['last_new_job'] = working_experience['last_new_job'].fillna(working_experience['last_new_job'].mode()[0])

In [117]:
# Display all unique values in the 'experience' column to check the distinct categories
working_experience['experience'].unique()

array(['>20', '15', '5', '<1', '11', '13', '7', '17', '2', '16', '1', '4',
       '10', '14', '18', '19', '12', '3', '6', '9', '8', '20'],
      dtype=object)

In [118]:
# Replace '<1' with 0 and '>20' with 21, then convert the 'experience' column to integer type
working_experience['experience'] = (working_experience['experience'].replace({'<1': 0, '>20': 21}).astype(int))

In [119]:
# Display all unique values in the 'company_size' column to check the distinct categories
working_experience['company_size'].unique()

array(['unknown', '50-99', '<10', '10000+', '5000-9999', '1000-4999',
       '10/49', '100-500', '500-999'], dtype=object)

In [120]:
# Replace any '/' characters in the 'company_size' column with '-'
working_experience['company_size'] = working_experience['company_size'].str.replace('/','-')

In [121]:
# Replace '<10' with '0-10' and '10000+' with '10000-14999' in the 'company_size' column
working_experience['company_size'] = working_experience['company_size'].replace({'<10':'0-10','10000+':'10000-14999'})

In [122]:
# Display all unique values in the 'company_type' column to check the distinct categories
working_experience['company_type'].unique()

array(['unknown', 'Pvt Ltd', 'Funded Startup', 'Early Stage Startup',
       'Other', 'Public Sector', 'NGO'], dtype=object)

In [123]:
# Convert all text in the 'company_type' column to uppercase
working_experience['company_type'] = working_experience['company_type'].str.upper()

In [124]:
# Display all unique values in the 'last_new_job' column to check the distinct categories
working_experience['last_new_job'].unique()

array(['1', '>4', 'never', '4', '3', '2'], dtype=object)

In [125]:
# Replace '>4' with '5' and 'never' with '0' in the 'last_new_job' column
working_experience['last_new_job'] = working_experience['last_new_job'].replace({'>4':'5','never':'0'})

In [126]:
# Automatically infer and convert the data types of all columns to the most appropriate types
working_experience = working_experience.convert_dtypes()

In [127]:
# Double-check the data after cleaning
working_experience.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   enrollee_id          19158 non-null  Int64 
 1   relevent_experience  19158 non-null  string
 2   experience           19158 non-null  Int64 
 3   company_size         19158 non-null  string
 4   company_type         19158 non-null  string
 5   last_new_job         19158 non-null  string
dtypes: Int64(2), string(4)
memory usage: 935.6 KB


## 4. Training hours

From LMS system's database you can retrieve a number of training hours for each student that they have completed.

Database credentials:

- Database type: MySQL
- Host: 112.213.86.31
- Port: 3360
- Login: etl_practice
- Password: 550814
- Database name: company_course
- Table name: training_hours

In [128]:
training_hours = read_data('sql', user_name='etl_practice', pw='550814', server='112.213.86.31', port='3360', db_name='company_course', table_name='training_hours')

In [140]:
# Display the first 5 rows of the DataFrame to quickly preview the data
training_hours.head()

,enrollee_id,training_hours
0,8949,36
1,29725,47
2,11561,83
3,33241,52
4,666,8


In [130]:
# Display a summary of the 'working_experience' DataFrame, including the number of non-null entries, column types
training_hours.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   enrollee_id     19158 non-null  int64
 1   training_hours  19158 non-null  int64
dtypes: int64(2)
memory usage: 299.5 KB


## 5. City development index

Another source that can be usefull is the table of City development index.

The City Development Index (CDI) is a measure designed to capture the level of development in cities. It may be significant for the resulting prediction of student's employment motivation.

It is stored here: https://sca-programming-school.github.io/city_development_index/index.html

In [131]:
city_index = read_data('html','https://sca-programming-school.github.io/city_development_index/index.html',table_index=0)

In [141]:
city_index.head().to_markdown()

'|    | City     |   City Development Index |\n|---:|:---------|-------------------------:|\n|  0 | city_103 |                    0.92  |\n|  1 | city_40  |                    0.776 |\n|  2 | city_21  |                    0.624 |\n|  3 | city_115 |                    0.789 |\n|  4 | city_162 |                    0.767 |'

In [133]:
# Display a summary of the 'first_table' DataFrame, including the number of non-null entries, column types
city_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   City                    123 non-null    object 
 1   City Development Index  123 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.1+ KB


## 6. Employment

From LMS database you can also retrieve the fact of employment. If student is marked as employed, it means that this student started to work in our company after finishing the course.

Database credentials:

- Database type: MySQL
- Host: 112.213.86.31
- Port: 3360
- Login: etl_practice
- Password: 550814
- Database name: company_course
- Table name: employment

In [134]:
employment = read_data('sql', user_name='etl_practice', pw='550814', server='112.213.86.31', port='3360', db_name='company_course',table_name='employment')

In [143]:
# Display the first 5 rows of the DataFrame to quickly preview the data
employment.head()

,enrollee_id,employed
0,1,0.0
1,2,1.0
2,4,0.0
3,5,0.0
4,7,0.0


In [136]:
# Display a summary of the 'first_table' DataFrame, including the number of non-null entries, column types
employment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   enrollee_id  19158 non-null  int64  
 1   employed     19158 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 299.5 KB


# LOAD DATA INTO DATABASE

In [137]:
# Define the SQLite database file path
db_name = "enrollee_datawarehouse.db"
# Create an SQLite database connection engine
engine = create_engine(f'sqlite:///{db_name}')
# Save each DataFrame to the SQLite database, replacing the existing tables if they exist
enrollies_data.to_sql("enrollies_data", engine,if_exists="replace")
enrollies_education.to_sql("enrollies_education", engine,if_exists="replace")
working_experience.to_sql("working_experience", engine, if_exists="replace")
training_hours.to_sql("training_hours", engine,if_exists="replace")
city_index.to_sql("city_index", engine,if_exists="replace")
employment.to_sql("employment", engine,if_exists="replace")


19158